In [ ]:
import requests
import json
import math
import time
import glob
import re

import numpy as np
import pandas as pd

# from datetime import datetime

In [ ]:
files = glob.glob(r'data/videos/*.xlsx')
files.sort()
files

In [ ]:
def getComments(videos_df):
    
    get_comments_url = "https://tiktok-scraper2.p.rapidapi.com/video/comments"
    headers = {
        "X-RapidAPI-Key": "YourOwnKey",
        "X-RapidAPI-Host": "YourOwnHost"
    }

    c_video_id_l = []
    comment_id_l = []
    comment_timestamp_l = []
    comment_text_l = []
    
    for i,video_url in enumerate(videos_df['video_url']):
          
        pages = math.ceil(videos_df['comments_count'][i] / 50)
      
        # on each page, get information of each comment and save it to the lists created above
        for page in range(pages):
            querystring = {"video_url":video_url,"count":"50","cursor":page*50}
            response_comments = requests.request("GET", get_comments_url, headers=headers, params=querystring)
            time.sleep(1)
            json_comments = json.loads(response_comments.text)
            
            try:
                comment_items = json_comments['comments']
                # cursor = json_comments['cursor']
                
                # get information of each comment (video id, comment id, time, text)
                for comment in comment_items:
                    c_video_id = comment['aweme_id']
                    comment_id = comment['cid']
                    comment_timestamp = comment['create_time']
                    comment_text = comment['text']
                    
                    # add comment information to each list
                    c_video_id_l.append(c_video_id)
                    comment_id_l.append(comment_id)
                    comment_timestamp_l.append(comment_timestamp)
                    comment_text_l.append(comment_text)
                        
            except:
                continue
    
            if json_comments == {}:
                break
                
            if json_comments['has_more'] == 0:
                break
                
    # save all comments information into a dictionary
    comment_dic = {'c_video_id': c_video_id_l,
                   'comment_id': comment_id_l,
                   'comment_timestamp': comment_timestamp_l,
                   'comment_text': comment_text_l
                  }
    
    # transfer to dataframe
    comment_df = pd.DataFrame(comment_dic)
    
    # change long numbers to string format so the value will not be changed in the excel files
    comment_df = comment_df.astype({"c_video_id": str, 'comment_id': str, 'comment_timestamp':str})
    
    return comment_df
    
        

In [ ]:
for file in files:
    
    # get hashtag name from the file name
    hashtag_name = re.findall('([a-z0-9]+)\.xlsx',file)[0]
    
    videos_df = pd.read_excel(file)
    
    comment_df = getComments(videos_df)
    
    # save comments table to local files
    comment_df.to_excel("data/comments/comments_{}.xlsx".format(hashtag_name), sheet_name='comments', index=False) 
    
    